# Speech Transcription on IPUs using Whisper - Inference

This notebook demonstrates speech transcription on the IPU using the [Whisper implementation in the Hugging Face Transformers library](https://huggingface.co/spaces/openai/whisper) alongside [Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).

Whisper is a versatile speech recognition model that can transcribe speech as well as perform multi-lingual translation and recognition tasks.
It was trained on diverse datasets to give human-level speech recognition performance without the need for fine tuning. 

Optimum Graphcore is the interface between the Hugging Face Transformers library and [Graphcore IPUs](https://www.graphcore.ai/products/ipu).
It provides a set of tools enabling model parallelization and loading on IPUs, training and fine-tuning on all the tasks already supported by Transformers while being compatible with the Hugging Face Hub and every model available on it out of the box.

> **Hardware requirements:** The Whisper models `whisper-tiny`, `whisper-base` and `whisper-small` can run two replicas on the smallest IPU-POD4 machine. The most capable model, `whisper-large`, will need to use either an IPU-POD16 or a Bow Pod16 machine. Please contact Graphcore if you'd like assistance running model sizes that don't work in this simple example notebook.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

In order to run this notebook you will need to be in an environment with the Poplar SDK installed and enabled. This is done by default on Paperspace. If you are not using Paperspace, refer to the [getting started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for a description of how to set this up.

We also need the Optimum Graphcore interface to the Hugging Face Transformers library, and there are a few extra dependencies we need to be able to handle audio.

In [ ]:
%pip install "optimum-graphcore>=0.6, <0.7"
%pip install soundfile==0.12.1 librosa==0.10.0.post2 tokenizers==0.12.1
%pip install matplotlib
%matplotlib inline

In [ ]:
%pip install examples-utils[common]@git+https://github.com/graphcore/examples-utils@latest_stable
from examples_utils import notebook_logging
%load_ext gc_logger

## Running Whisper on the IPU

We start by importing the required modules, some of which are needed to configure the IPU.


In [ ]:
# Generic imports
from datasets import load_dataset
import matplotlib
import librosa
import IPython
import random
import os

# IPU-specific imports
from optimum.graphcore import IPUConfig
from optimum.graphcore.modeling_utils import to_pipelined

# HF-related imports
from transformers import WhisperProcessor, WhisperForConditionalGeneration

The Whisper model is available on Hugging Face in several sizes, from `whisper-tiny` with 39M parameters to `whisper-large` with 1550M parameters.

We download `whisper-tiny` which we will run using two IPUs.
The [Whisper architecture](https://openai.com/research/whisper) is an encoder-decoder Transformer, with the audio split into 30-second chunks.
For simplicity one IPU is used for the encoder part of the graph and another for the decoder part.
The `IPUConfig` object helps to configure the model to be pipelined across the IPUs.

In [ ]:
model_spec = "openai/whisper-tiny.en"

# Instantiate processor and model
processor = WhisperProcessor.from_pretrained(model_spec)
model = WhisperForConditionalGeneration.from_pretrained(model_spec)

# Adapt whisper-tiny to run on the IPU
ipu_config = IPUConfig(ipus_per_replica=2, executable_cache_dir="./exe_cache")
pipelined_model = to_pipelined(model, ipu_config)
pipelined_model = pipelined_model.parallelize(for_generation=True).half()

Now we can load the dataset and process an example audio file.
If precompiled models are not available, then the first run of the model triggers two graph compilations.
This means that our first test transcription could take a minute or two to run, but subsequent runs will be much faster.

In [ ]:
# load the dataset and read an example sound file
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
test_sample = ds[2]
sample_rate = test_sample['audio']['sampling_rate']

def transcribe(data, rate):
    input_features = processor(data, return_tensors="pt", sampling_rate=rate).input_features.half()

    # This triggers a compilation, unless a precompiled model is available.
    sample_output = pipelined_model.generate(input_features, max_length=448, min_length=3)
    transcription = processor.batch_decode(sample_output, skip_special_tokens=True)[0]
    return transcription

test_transcription = transcribe(test_sample["audio"]["array"], sample_rate)

In the next cell, we compare the expected text from the dataset with the transcribed result from the model.
There will typically be some small differences, but even `whisper-tiny` does a great job! It even adds punctuation.

You can listen to the audio and compare the model result yourself using the controls below.

In [ ]:
print(f"Expected: {test_sample['text']}\n")
print(f"Transcribed: {test_transcription}")

matplotlib.pyplot.figure(figsize=(14, 5))
librosa.display.waveshow(test_sample["audio"]["array"], sr=sample_rate)
IPython.display.Audio(test_sample["audio"]["array"], rate=sample_rate)
print(sample_rate)

The model only needs to be compiled once. Subsequent inferences will be much faster.
In the cell below, we repeat the exercise but with a random example from the dataset.

You might like to re-run this next cell multiple times to get different comparisons.

In [ ]:
idx = random.randint(0, ds.num_rows - 1)
data = ds[idx]["audio"]["array"]

print(f"Example #{idx}\n")
print(f"Expected: {ds[idx]['text']}\n")
print(f"Transcribed: {transcribe(data, sample_rate)}")

matplotlib.pyplot.figure(figsize=(14, 5))
librosa.display.waveshow(data, sr=sample_rate)
IPython.display.Audio(data, rate=sample_rate, autoplay=True)


Finally, we detach the process from the IPUs when we are done to make the IPUs available to other users.

In [ ]:
pipelined_model.detachFromDevice()

## Next Steps

The `whisper-tiny` model used here is very fast for inference and so cheap to run, but its accuracy can be improved.
The `whisper-base` and `whisper-small` models have 74M and 244M parameters respectively (compared to just 39M for `whisper-tiny`). You can try out `whisper-base` and `whisper-small` by changing `model_spec = "openai/whisper-tiny.en"` (at the beginning of this notebook) to `model_spec = "openai/whisper-base.en"` or `model_spec = "openai/whisper-small.en"` respectively.

Larger models and multilingual models are also available.
To access the multilingual models, remove the `.en` from the checkpoint name. Note however that the multilingual models are slightly less accurate for this English transcription task but they can be used for transcribing other languages or for translating to English.

The largest models have 1550M parameters and won't fit with our simple two-IPU pipeline.
To run these you will need more than the IPU-POD4. On Paperspace, this is available using either an IPU-POD16 or a Bow Pod16 machine. Please contact Graphcore if you need assistance running these larger models.


## Conclusion

In this notebook we demonstrated using Whisper for speech recognition and transcription on the IPU.
We used the Optimum Graphcore package to interface between the IPU and the Hugging Face Transformers library. This meant that only a few lines of code were needed to get this state-of-the-art automated speech recognition model running on IPUs.

# Bonus: Deploy whisper in one-click
We can now deploy our own Whisper API hosted in Paperspace Deployment!
Find a step-by-step guide in [deploy-whisper-api notebook](deploy-whisper-api.ipynb)

In [48]:
%load_ext autoreload
%autoreload 2
from widgets import deploy,shutdown

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
deploy()

**Once you are done, you can close the server to free the resources**

In [49]:
shutdown()